In [ ]:
## De functie die tf-records leest en ze uiteindelijk weer wegschrijft (maar dan met minder labels) heet readTfRecord
## Deze functie roept andere functies aan: getLabels en getLabelIndices

import numpy as np
import pandas as pd
import re
import os
import random
from collections import Counter

import tensorflow as tf
from tensorflow import gfile

In [ ]:
## Paden naar originele tf-records: bal_train, unbal_train en eval
bal_train_data_pattern = "tfrecords/bal_train/*.tfrecord"
unbal_train_data_pattern = "tfrecords/unbal_train/*.tfrecord"
eval_data_pattern = "tfrecords/eval/*.tfrecord"

In [ ]:
## Oorspronkelijke csv-file
mid_to_label_old = pd.read_csv("csv_files/class_labels_indices.csv", sep=",")
target_label = 500 # silence

In [ ]:
def findTfRecord(tfrecords_filename):
    
    ## Maak 2 writers (train en test) die nieuw tf-records schrijft
    
    record_name = tfrecords_filename.split('\\')[-1]
  
    ## Iterator that goes through all examples of tf-record
    record_iterator = tf.python_io.tf_record_iterator(path=tfrecords_filename)

    
    for string_record in record_iterator:
        
        ## Take one example
        example = tf.train.SequenceExample()
        example.ParseFromString(string_record)
 
        ## Get labels and find for each label the m-code in class_label_indices_old
        labels = example.context.feature['labels'].int64_list.value
        
        if len(labels)==1 and label[0] == target_label:
            print(record_name)
            print(example)

In [ ]:
## Maak nieuwe tf-records van bal_train

files = gfile.Glob(bal_train_data_pattern)

for file in files:
    ## Read a file and rewrite it
    readWriteTfRecord(file, 'B')

In [ ]:
## Maak nieuwe tf-records van unbal_train

files = gfile.Glob(unbal_train_data_pattern)

for file in files:
    ## Read a file and rewrite it
    readWriteTfRecord(file, 'U')

In [ ]:
## Maak nieuwe tf-records van eval

files = gfile.Glob(eval_data_pattern)

for file in files:
    
    ## Read a file and rewrite it
    readWriteTfRecord(file, 'E')

In [ ]:
## Print voor alle labels het aantal voorkomens in de nieuwe tf-records

sum_occur = 0
cnt_labels = Counter(all_labels)

for i in cnt_labels.most_common(100):
    sum_occur = sum_occur + i[1]
    print(label_proportions.loc[label_proportions['index']==i[0]]['display_name'].values[0], ' : ', str(i[1]))

In [ ]:
## Tellen van lijsten in lijsten is lastig, vandaar dat ik er strings van maak: [0,4] wordt ["0_4"]
str_combis = []

for combi in label_combinations:
    str_combi = [str(i) for i in combi]
    str_combi = "_".join(str_combi)
    str_combis.append(str_combi)

In [ ]:
## Houd individuele voorkomens bij (dus aantal keer dat label voorkomt zonder andere labels)
ind_occur = [0 for x in range(label_proportions.shape[0])] 

## Geef meest voorkomende combinaties
cnt_label_combinations = Counter(str_combis)
for label_combination in cnt_label_combinations.most_common(20):
    label_names = []
    
    ## Maak van string weer een lijst (zie vorige cel)
    label_list = label_combination[0].split("_")
    
    ## Code die alleen individuele labels telt
    if(len(label_list)==1):
        ind_occur[int(label_list[0])] = label_combination[1]
    
    for i in label_list:
        label_names.append(str(label_proportions.loc[label_proportions['index']==i]['display_name']))
    label_names = ', '.join(label_names)
    print(label_names,': ', label_combination[1])

In [ ]:
print("sum occurences: " + str(sum_occur))
print("ind occ: ")
print(ind_occur)

In [ ]:
## UP-/DOWN-SAMPLEN

## Alle code hierna is alleen om uit te rekenen wat de proporties zijn
## Het werkt als volgt:
## Van het totale aantal voorkomens van een label (bv 11.000), wordt het gewenste afgetrokken:
## bv 11.000-2.000 = 9.000
## Dit aantal examples moet geskipt worden, maar dan alleen als het label alleen voorkomt
## Daarom wordt de target proportie van het totale individuutjes (target_ind) als volgt berekend:
## Stel ind_occ (aantal keer alleen voorkomend) is 10.000, waarvan we er 9.000 niet willen hebben.
## (10.000-9.000)/10.000 = 0.1 is dan de kans dat een example wordt geschreven.

In [ ]:
## Code die kolom toevoegt aan label-csv (proportion) en deze in de forloop vult

## Aantal gewenste samples per label: 2500
target = 2500

## Aantal voorkomens en aantal voorkomens alleen
label_proportions["total"] = 0
label_proportions["ind_total"] = 0
label_proportions["proportion"] = 0
label_proportions["target_ind"] = 0

for i in range(mid_to_label_new.shape[0]):
    label_proportions.loc[i,"total"] = cnt_labels[i]
    label_proportions.loc[i,"ind_total"] = ind_occur[i]   
    label_proportions.loc[i,"proportion"] = cnt_labels[i]/sum_occur
    label_proportions.loc[i,"target_ind"] = (ind_occur[i]-(cnt_labels[i]-target))/ind_occur[i]

In [ ]:
## Schrijf bestandje weg
label_proportions.to_csv(name_label_prop_file, sep = ";")

In [ ]:
label_proportions